In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import random
import pandas as pd

In [2]:
Movies = pd.read_csv('Movielist_DC.csv')
Movies['name_link'][10] = '1001781-batman'
Movies['name_link'][16] = '1077027-batman_and_robin'
Movies['name_link'][17] = '1078035-steel'
Movies['name_link'][32] = '1226860-losers'
Movies

/nfshome/jc9033/.conda/envs/myEnv/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/nfshome/jc9033/.conda/envs/myEnv/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/nfshome/jc9033/.conda/envs/myEnv/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after remo

,Unnamed: 0,name,name_link
0,0,Superman and the Mole-Men,superman-and-the-molemen
1,1,Batman: The Movie,batman_the_movie
2,2,Wonder Woman(2009),wonder-woman
3,3,Superman,superman_the_movie
4,4,Superman II,superman_ii
5,5,Swamp Thing,swamp_thing
6,6,Superman III,superman_iii
7,7,Supergirl,supergirl
8,8,Superman IV: The Quest for Peace,superman_iv_the_quest_for_peace
9,9,The Return of Swamp Thing,return_of_swamp_thing


In [3]:
user_agents = ['Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1',
               'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50',
               'Opera/9.80 (Windows NT 6.1; U; en) Presto/2.8.131 Version/11.11']

In [4]:
def find_star_missing(soup):
    # find the indexes for star missing
    star_missing = []
    review_frame=soup.find_all('div',class_='col-xs-16')
    for i in range(len(review_frame)):
        if not review_frame[i].find('span', class_="glyphicon glyphicon-star") and not review_frame[i].find(text = re.compile('\xbd')):
            star_missing.append(i)
    return star_missing

def crawler(url):
    # fetch html by url
    headers = {'User-Agent':random.choice(user_agents)}
    r = requests.get(url, timeout=(3.05, 10), headers = headers)
    soup = BeautifulSoup(r.text, 'html.parser')

    # parse html
    try:
        data = []
        user_review=soup.find_all('div',class_='user_review')
        stars=soup.find_all('span',class_='fl')
        date=soup.find_all('span',class_='fr small subtle')
        for i in range(len(user_review)):        
            stars_single = len(stars[i].find_all('span', class_="glyphicon glyphicon-star")) + float(len(stars[i].text)-1)/2            
            data.append((user_review[i].text, stars_single, date[i].text))
            
    except: # star missing
        data = []
        user_review=soup.find_all('div',class_='user_review')
        stars=soup.find_all('span',class_='fl')
        date=soup.find_all('span',class_='fr small subtle')
        
        # insert star missing
        star_missing = find_star_missing(soup)
        for insert_index in star_missing:    
            stars.insert(insert_index,0)
            
        for i in range(len(user_review)):
            if stars[i] == 0:
                stars_single = 0
            else:
                stars_single = len(stars[i].find_all('span', class_="glyphicon glyphicon-star")) + float(len(stars[i].text)-1)/2             
            data.append((user_review[i].text, stars_single, date[i].text))
            
    return data

In [5]:
for name_link in Movies['name_link']:
    index = Movies.index[Movies['name_link'] == name_link].tolist()[0]
    data = []
    # 1st page
    url = 'https://www.rottentomatoes.com/m/' + name_link + '/reviews/?type=user&sort='
    data = crawler(url)
    print(name_link, "page%d"%1, "Total reviews:%d"%len(data))
    # 2-51st page
    for i in range(2,52):
        attempts = 0
        success = False
        while attempts < 3 and not success:    
            try:
                time.sleep(random.uniform(1, 2))
                url = 'https://www.rottentomatoes.com/m/' + name_link + '/reviews/?page='+ str(i) + '&type=user&sort='
                data_page = crawler(url)

                data = data + data_page
                print(name_link, "page%d"%i, "Total reviews:%d"%len(data_page))
                success = True
            except:
                attempts += 1
                if attempts == 3:
                    print(i, 'error')
        if len(data_page) == 0:
            print('The number of reviews is less than 1000!')
            break
    # output
    df = pd.DataFrame(data, columns=['review', 'stars', 'date'])
    df.to_csv('Reviews_DC/'+Movies['name'][index]+'.csv', index=False, encoding='utf-8')
    print(name_link, "Finished task %d"%index, "Total reviews:%d"%len(df))

('superman-and-the-molemen', 'page1', 'Total reviews:20')
('superman-and-the-molemen', 'page2', 'Total reviews:14')
('superman-and-the-molemen', 'page3', 'Total reviews:0')
The number of reviews is less than 1000!
('superman-and-the-molemen', 'Finished task 0', 'Total reviews:34')
('batman_the_movie', 'page1', 'Total reviews:19')
('batman_the_movie', 'page2', 'Total reviews:20')
('batman_the_movie', 'page3', 'Total reviews:20')
('batman_the_movie', 'page4', 'Total reviews:19')
('batman_the_movie', 'page5', 'Total reviews:20')
('batman_the_movie', 'page6', 'Total reviews:20')
('batman_the_movie', 'page7', 'Total reviews:20')
('batman_the_movie', 'page8', 'Total reviews:20')
('batman_the_movie', 'page9', 'Total reviews:20')
('batman_the_movie', 'page10', 'Total reviews:20')
('batman_the_movie', 'page11', 'Total reviews:20')
('batman_the_movie', 'page12', 'Total reviews:20')
('batman_the_movie', 'page13', 'Total reviews:20')
('batman_the_movie', 'page14', 'Total reviews:20')
('batman_the_

('superman_ii', 'page51', 'Total reviews:20')
('superman_ii', 'Finished task 4', 'Total reviews:1006')
('swamp_thing', 'page1', 'Total reviews:20')
('swamp_thing', 'page2', 'Total reviews:19')
('swamp_thing', 'page3', 'Total reviews:20')
('swamp_thing', 'page4', 'Total reviews:20')
('swamp_thing', 'page5', 'Total reviews:20')
('swamp_thing', 'page6', 'Total reviews:20')
('swamp_thing', 'page7', 'Total reviews:19')
('swamp_thing', 'page8', 'Total reviews:20')
('swamp_thing', 'page9', 'Total reviews:20')
('swamp_thing', 'page10', 'Total reviews:19')
('swamp_thing', 'page11', 'Total reviews:20')
('swamp_thing', 'page12', 'Total reviews:20')
('swamp_thing', 'page13', 'Total reviews:19')
('swamp_thing', 'page14', 'Total reviews:20')
('swamp_thing', 'page15', 'Total reviews:20')
('swamp_thing', 'page16', 'Total reviews:20')
('swamp_thing', 'page17', 'Total reviews:20')
('swamp_thing', 'page18', 'Total reviews:20')
('swamp_thing', 'page19', 'Total reviews:20')
('swamp_thing', 'page20', 'Total

('superman_iv_the_quest_for_peace', 'page31', 'Total reviews:20')
('superman_iv_the_quest_for_peace', 'page32', 'Total reviews:20')
('superman_iv_the_quest_for_peace', 'page33', 'Total reviews:20')
('superman_iv_the_quest_for_peace', 'page34', 'Total reviews:19')
('superman_iv_the_quest_for_peace', 'page35', 'Total reviews:20')
('superman_iv_the_quest_for_peace', 'page36', 'Total reviews:19')
('superman_iv_the_quest_for_peace', 'page37', 'Total reviews:20')
('superman_iv_the_quest_for_peace', 'page38', 'Total reviews:20')
('superman_iv_the_quest_for_peace', 'page39', 'Total reviews:20')
('superman_iv_the_quest_for_peace', 'page40', 'Total reviews:20')
('superman_iv_the_quest_for_peace', 'page41', 'Total reviews:20')
('superman_iv_the_quest_for_peace', 'page42', 'Total reviews:20')
('superman_iv_the_quest_for_peace', 'page43', 'Total reviews:20')
('superman_iv_the_quest_for_peace', 'page44', 'Total reviews:20')
('superman_iv_the_quest_for_peace', 'page45', 'Total reviews:20')
('superman

('batman_mask_of_the_phantasm', 'page20', 'Total reviews:20')
('batman_mask_of_the_phantasm', 'page21', 'Total reviews:20')
('batman_mask_of_the_phantasm', 'page22', 'Total reviews:20')
('batman_mask_of_the_phantasm', 'page23', 'Total reviews:20')
('batman_mask_of_the_phantasm', 'page24', 'Total reviews:19')
('batman_mask_of_the_phantasm', 'page25', 'Total reviews:20')
('batman_mask_of_the_phantasm', 'page26', 'Total reviews:20')
('batman_mask_of_the_phantasm', 'page27', 'Total reviews:20')
('batman_mask_of_the_phantasm', 'page28', 'Total reviews:20')
('batman_mask_of_the_phantasm', 'page29', 'Total reviews:20')
('batman_mask_of_the_phantasm', 'page30', 'Total reviews:20')
('batman_mask_of_the_phantasm', 'page31', 'Total reviews:20')
('batman_mask_of_the_phantasm', 'page32', 'Total reviews:20')
('batman_mask_of_the_phantasm', 'page33', 'Total reviews:20')
('batman_mask_of_the_phantasm', 'page34', 'Total reviews:20')
('batman_mask_of_the_phantasm', 'page35', 'Total reviews:19')
('batman

('1077027-batman_and_robin', 'page18', 'Total reviews:20')
('1077027-batman_and_robin', 'page19', 'Total reviews:20')
('1077027-batman_and_robin', 'page20', 'Total reviews:19')
('1077027-batman_and_robin', 'page21', 'Total reviews:19')
('1077027-batman_and_robin', 'page22', 'Total reviews:20')
('1077027-batman_and_robin', 'page23', 'Total reviews:20')
('1077027-batman_and_robin', 'page24', 'Total reviews:20')
('1077027-batman_and_robin', 'page25', 'Total reviews:20')
('1077027-batman_and_robin', 'page26', 'Total reviews:20')
('1077027-batman_and_robin', 'page27', 'Total reviews:18')
('1077027-batman_and_robin', 'page28', 'Total reviews:20')
('1077027-batman_and_robin', 'page29', 'Total reviews:20')
('1077027-batman_and_robin', 'page30', 'Total reviews:20')
('1077027-batman_and_robin', 'page31', 'Total reviews:20')
('1077027-batman_and_robin', 'page32', 'Total reviews:20')
('1077027-batman_and_robin', 'page33', 'Total reviews:20')
('1077027-batman_and_robin', 'page34', 'Total reviews:20

...

('the_spirit', 'page17', 'Total reviews:20')
('the_spirit', 'page18', 'Total reviews:20')
('the_spirit', 'page19', 'Total reviews:20')
('the_spirit', 'page20', 'Total reviews:20')
('the_spirit', 'page21', 'Total reviews:20')
('the_spirit', 'page22', 'Total reviews:20')
('the_spirit', 'page23', 'Total reviews:19')
('the_spirit', 'page24', 'Total reviews:19')
('the_spirit', 'page25', 'Total reviews:20')
('the_spirit', 'page26', 'Total reviews:20')
('the_spirit', 'page27', 'Total reviews:20')
('the_spirit', 'page28', 'Total reviews:20')
('the_spirit', 'page29', 'Total reviews:20')
('the_spirit', 'page30', 'Total reviews:20')
('the_spirit', 'page31', 'Total reviews:18')
('the_spirit', 'page32', 'Total reviews:20')
('the_spirit', 'page33', 'Total reviews:20')
('the_spirit', 'page34', 'Total reviews:19')
('the_spirit', 'page35', 'Total reviews:20')
('the_spirit', 'page36', 'Total reviews:20')
('the_spirit', 'page37', 'Total reviews:19')
('the_spirit', 'page38', 'Total reviews:19')
('the_spir

('jonah_hex', 'page42', 'Total reviews:19')
('jonah_hex', 'page43', 'Total reviews:20')
('jonah_hex', 'page44', 'Total reviews:19')
('jonah_hex', 'page45', 'Total reviews:19')
('jonah_hex', 'page46', 'Total reviews:19')
('jonah_hex', 'page47', 'Total reviews:20')
('jonah_hex', 'page48', 'Total reviews:20')
('jonah_hex', 'page49', 'Total reviews:19')
('jonah_hex', 'page50', 'Total reviews:20')
('jonah_hex', 'page51', 'Total reviews:19')
('jonah_hex', 'Finished task 33', 'Total reviews:1001')
('red', 'page1', 'Total reviews:19')
('red', 'page2', 'Total reviews:19')
('red', 'page3', 'Total reviews:20')
('red', 'page4', 'Total reviews:19')
('red', 'page5', 'Total reviews:20')
('red', 'page6', 'Total reviews:20')
('red', 'page7', 'Total reviews:20')
('red', 'page8', 'Total reviews:20')
('red', 'page9', 'Total reviews:20')
('red', 'page10', 'Total reviews:20')
('red', 'page11', 'Total reviews:20')
('red', 'page12', 'Total reviews:20')
('red', 'page13', 'Total reviews:19')
('red', 'page14', '

('superman_man_of_steel', 'page7', 'Total reviews:20')
('superman_man_of_steel', 'page8', 'Total reviews:20')
('superman_man_of_steel', 'page9', 'Total reviews:20')
('superman_man_of_steel', 'page10', 'Total reviews:20')
('superman_man_of_steel', 'page11', 'Total reviews:20')
('superman_man_of_steel', 'page12', 'Total reviews:20')
('superman_man_of_steel', 'page13', 'Total reviews:20')
('superman_man_of_steel', 'page14', 'Total reviews:19')
('superman_man_of_steel', 'page15', 'Total reviews:20')
('superman_man_of_steel', 'page16', 'Total reviews:20')
('superman_man_of_steel', 'page17', 'Total reviews:20')
('superman_man_of_steel', 'page18', 'Total reviews:20')
('superman_man_of_steel', 'page19', 'Total reviews:19')
('superman_man_of_steel', 'page20', 'Total reviews:20')
('superman_man_of_steel', 'page21', 'Total reviews:20')
('superman_man_of_steel', 'page22', 'Total reviews:20')
('superman_man_of_steel', 'page23', 'Total reviews:20')
('superman_man_of_steel', 'page24', 'Total reviews:

('suicide_squad_2016', 'page1', 'Total reviews:20')
('suicide_squad_2016', 'page2', 'Total reviews:20')
('suicide_squad_2016', 'page3', 'Total reviews:20')
('suicide_squad_2016', 'page4', 'Total reviews:20')
('suicide_squad_2016', 'page5', 'Total reviews:20')
('suicide_squad_2016', 'page6', 'Total reviews:20')
('suicide_squad_2016', 'page7', 'Total reviews:20')
('suicide_squad_2016', 'page8', 'Total reviews:20')
('suicide_squad_2016', 'page9', 'Total reviews:20')
('suicide_squad_2016', 'page10', 'Total reviews:20')
('suicide_squad_2016', 'page11', 'Total reviews:20')
('suicide_squad_2016', 'page12', 'Total reviews:20')
('suicide_squad_2016', 'page13', 'Total reviews:20')
('suicide_squad_2016', 'page14', 'Total reviews:19')
('suicide_squad_2016', 'page15', 'Total reviews:20')
('suicide_squad_2016', 'page16', 'Total reviews:20')
('suicide_squad_2016', 'page17', 'Total reviews:20')
('suicide_squad_2016', 'page18', 'Total reviews:20')
('suicide_squad_2016', 'page19', 'Total reviews:20')
('

('wonder_woman_2017', 'page50', 'Total reviews:20')
('wonder_woman_2017', 'page51', 'Total reviews:20')
('wonder_woman_2017', 'Finished task 42', 'Total reviews:1013')
('justice_league_2017', 'page1', 'Total reviews:20')
('justice_league_2017', 'page2', 'Total reviews:20')
('justice_league_2017', 'page3', 'Total reviews:20')
('justice_league_2017', 'page4', 'Total reviews:20')
('justice_league_2017', 'page5', 'Total reviews:20')
('justice_league_2017', 'page6', 'Total reviews:20')
('justice_league_2017', 'page7', 'Total reviews:20')
('justice_league_2017', 'page8', 'Total reviews:19')
('justice_league_2017', 'page9', 'Total reviews:20')
('justice_league_2017', 'page10', 'Total reviews:20')
('justice_league_2017', 'page11', 'Total reviews:20')
('justice_league_2017', 'page12', 'Total reviews:20')
('justice_league_2017', 'page13', 'Total reviews:20')
('justice_league_2017', 'page14', 'Total reviews:20')
('justice_league_2017', 'page15', 'Total reviews:20')
('justice_league_2017', 'page16

# Merge csv files
https://www.cnblogs.com/taolusi/p/9292995.html

In [6]:
for i in Movies['name']:
    df = pd.read_csv('Reviews_DC/'+i+'.csv')
    df['name'] = i
    df.to_csv('Reviews_DC.csv', mode='a+', index=False, header=False)
    print(df.shape[0], i)

(34, 'Superman and the Mole-Men')
(1014, 'Batman: The Movie')
(22, 'Wonder Woman(2009)')
(1007, 'Superman')
(1006, 'Superman II')
(543, 'Swamp Thing')
(1012, 'Superman III')
(1008, 'Supergirl')
(1014, 'Superman IV: The Quest for Peace')
(114, 'The Return of Swamp Thing')
(1007, 'Batman')
(1012, 'Batman Returns')
(1008, 'Batman: Mask of the Phantasm')
(1018, 'Tank Girl')
(1011, 'Batman Forever')
(25, 'Justice League of America')
(1009, 'Batman & Robin')
(340, 'Steel')
(13, 'Gen 13')
(1012, 'Road to Perdition')
(1012, 'The League of Extraordinary Gentlemen')
(1014, 'American Splendor')
(1008, 'Catwoman')
(1006, 'Constantine')
(1010, 'V for Vendetta')
(1012, 'Batman Begins')
(1013, 'A History of Violence')
(1016, 'Superman Returns')
(1019, 'Stardust')
(1011, 'The Dark Knight')
(1005, 'The Spirit')
(1012, 'Watchmen')
(1010, 'The Losers')
(1001, 'Jonah Hex')
(1011, 'RED')
(1014, 'Green Lantern')
(1012, 'The Dark Knight Rises')
(1015, 'Man of Steel')
(1016, 'RED 2')
(1013, 'Batman v Superman

In [7]:
df = pd.read_csv('Reviews_DC/'+i+'.csv')
df.head()

,review,stars,date
0,I saw this movie with my brother and my dad ...,1.5,"April 13, 2019"
1,Did everyone see the same movie I saw? This ...,1.5,"April 13, 2019"
2,"Shazam!, like its titular superhero takes a ...",4.0,"April 13, 2019"
3,such a fun hearted with lot of humour pumpin...,5.0,"April 13, 2019"
4,The movie was really amazing! Love where DC ...,5.0,"April 13, 2019"


In [8]:
# add header & column order
df = pd.read_csv('Reviews_DC.csv', header=None).drop_duplicates()
df.columns = ['review', 'stars', 'date', 'name']
columns = ['name', 'review', 'stars', 'date']
df.to_csv('Reviews_DC.csv', index=False, columns=columns)
print(df.shape)
df.head()

(40385, 4)


,review,stars,date,name
0,"An average b-movie of the early 50s, precedi...",2.0,"April 6, 2019",Superman and the Mole-Men
1,About what you would expect. Not that much a...,2.0,"May 7, 2017",Superman and the Mole-Men
2,A charming little tale where the villains ar...,3.0,"June 15, 2015",Superman and the Mole-Men
3,"Where to even begin? For starters, this is m...",3.0,"June 8, 2015",Superman and the Mole-Men
4,terrible. superman is only in it for like 2 ...,0.5,"September 29, 2014",Superman and the Mole-Men


In [9]:
df = pd.read_csv('Reviews_DC.csv')
print(df.shape)
df.head()

(40385, 4)


,name,review,stars,date
0,Superman and the Mole-Men,"An average b-movie of the early 50s, precedi...",2.0,"April 6, 2019"
1,Superman and the Mole-Men,About what you would expect. Not that much a...,2.0,"May 7, 2017"
2,Superman and the Mole-Men,A charming little tale where the villains ar...,3.0,"June 15, 2015"
3,Superman and the Mole-Men,"Where to even begin? For starters, this is m...",3.0,"June 8, 2015"
4,Superman and the Mole-Men,terrible. superman is only in it for like 2 ...,0.5,"September 29, 2014"


In [10]:
len(df['name'].unique())

46